In [2]:
import matplotlib
import os
import inspect
import requests
import time
from tqdm.notebook import tqdm
import random
import pandas as pnd
from boardgamegeek import BoardGameGeek

## Step 1

Access BBG API's to download game and user preferences data. Define criterion used for setting the parameters that drive the download in termis of the variables defining the dataset, such collecting games only for a specified period or a specified set of categories. The wider the game collection, the better.


### Reading of latest dataset

In [3]:
df = pnd.read_csv("BGG_latest.csv")
df.shape

(18931, 9)

### Selecting timeframe of 20 years to be analyzed

In [4]:
df = df.loc[df["Year"].isin(range(2000, 2020))]
for col in df.columns:
    print(col)
print("\nShape: ", df.shape)
df.loc[df["ID"] == 34084]
#df_sample = df.sample(frac=0.6).reset_index(drop=True)
#df_sample = df

ID
Name
Year
Rank
Average
Bayes average
Users rated
URL
Thumbnail

Shape:  (14115, 9)


,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
851,34084,Witch's Brew,2008,852,7.03,6.587,3581,/boardgame/34084/witchs-brew,https://cf.geekdo-images.com/micro/img/H2bwR-T...


### Construction of new dataframe with notable columns
<ul>
	<li>Eventual implementation of "Error_row", a list to correct the errors in retrieval phase</li>
</ul>

In [5]:
bgg = BoardGameGeek()
g = bgg.game(name=None, game_id=34084)

In [7]:
cols = list(g.__dict__["_data"])
new_df = pnd.DataFrame(columns = cols)
new_df = new_df.drop(["thumbnail", "image", "expansion", "implementations", "expansions", "expands", 
             "alternative_names", "trading", "wanting", "wishing", "numcomments"], axis=1)
error_row = []
#new_df

#### Retrieval of row informations via API request
<ul>
    <li>Some errors in retrieval phase pop up. Have to fix them, may do it in finale phase.</li>
    <li>Could clean it, may do it later.</li>
</ul>

In [ ]:
for index, row in tqdm(list(df_sample.iterrows())):
    try:
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
    except:
        print("Error on iteration row {0}".format(index))
        #error_row.append(df_sample.iloc[index])
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
        pass
    time.sleep(0.5)

##### Correct retriever, modified in upper cell in order to try something more complex eventually. Make it "code" to try it

for index, row in tqdm(list(df_sample.iterrows())):
    try:
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
    except:
        print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass
    time.sleep(0.5)

##### Stub error solver, make it "code" to try it
while len(error_row) > 0:
    try:
        g = bgg.game(name=None, game_id=error_row[i])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
        error_row.pop(0)
    except:
        #print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass

##### Dataframe cleaner from duplicates and saving to csv

In [ ]:
new_df.drop_duplicates(subset="id", keep=False, inplace=True)
new_df.to_csv("./BGG_sampled.csv")

## TODO: Make a python class in order to handle all this operation cleanly